In [1]:
import time
import numpy as np
import time
import pandas as pd
import re
from textblob import TextBlob
import datetime

In [2]:

start_time = time.time()



# this feature will take the "STATE/REGION" and "COUNTRY/REGION" from hubspot
# clean them from punctuatuion and wrongfuly imported data
# output: Record ID, STATE/REGION (cleaned) , COUNTRY/REGION (cleaned), Continent (created based on the state and country)

#Input option 1 - Redshift table synced from HS "hs_contacts"
#Input option 2 - Exported CSV from HubSpot of Record ID, STATE/REGION, COUNTRY/REGION
#select only Record ID, STATE/REGION, COUNTRY/REGION 
"""

import awswrangler as wr

con_redshift = wr.redshift.connect("redshift-cluster-matil")

hs_contacts = wr.redshift.read_sql_query("SELECT * FROM public.hs_contacts", con=con_redshift)


"""

#func that will remove letters that are not in english



def remove_diacritics(input_str):



    # Replace English letters with diacritics with their corresponding plain English letters
    input_str = re.sub(r"ß", "ss", input_str)
    input_str = re.sub(r"Š|š", "s", input_str)
    input_str = re.sub(r"Ž|ž", "z", input_str)
    input_str = re.sub(r"À|Á|Â|Ã|Ä|Å|à|á|â|ã|ä|å|æ", "a", input_str)
    input_str = re.sub(r"æ|Æ", "ae", input_str)
    input_str = re.sub(r"Ç|ç|Ç", "C", input_str)
    input_str = re.sub(r"È|É|Ê|Ë|è|é|ê|ë|ë", "e", input_str)
    input_str = re.sub(r"Ì|Í|Î|Ï|ì|í|î|ï", "i", input_str)
    input_str = re.sub(r"Ð", "d", input_str)
    input_str = re.sub(r"Ñ|ñ", "n", input_str)
    input_str = re.sub(r"ð|ò|ó|ô|õ|ö|ø|Ò|Ó|Ô|Õ|Ö|Ø", "o", input_str)
    input_str = re.sub(r"Ù|Ú|Û|Ü|ù|ú|û|ü", "u", input_str)
    input_str = re.sub(r"Ý|ÿ", "y", input_str)
    input_str = re.sub(r"Þ|þ", "th", input_str)
    input_str = re.sub(r"ß", "ss", input_str)
    # Replace non-English letters with spaces
    input_str = re.sub(r"[^a-zA-Z\s]", " ", input_str)


    
    return input_str.strip()




#change long state name to abv for canada
def replace_canada_state_name(row):
    if pd.isna(row['State/Region']):
        return row
    for state, abv in canada_states.items():
        if state in row['State/Region']:
            row['State/Region'] = abv
    return row



#check if there is a us state in the country column, move it to the correct column and fix the country name
def move_state_name(row):
    if row['Country/Region'] in us_state_to_abbrev.values():
        row['State/Region'] = row['Country/Region']
        row['Country/Region'] = 'us'
    return row



# Define a function to look up the continent for a given country
def get_continent(country):
    for continent, countries in countries_continents.items():
        if country in countries:
            return continent
    return None




# United States of America Python Dictionary to translate States,
# Districts & Territories to Two-Letter codes and vice versa.
#
# Canonical URL: https://gist.github.com/rogerallen/1583593
#
# Dedicated to the public domain.  To the extent possible under law,
# Roger Allen has waived all copyright and related or neighboring
# rights to this code.  Data originally from Wikipedia at the url:
# https://en.wikipedia.org/wiki/ISO_3166-2:US
#
# Automatically Generated 2021-09-11 18:04:36 via Jupyter Notebook from
# https://gist.github.com/rogerallen/d75440e8e5ea4762374dfd5c1ddf84e0 

us_state_to_abbrev = {
    "alabama": "al",
    "alaska": "ak",
    "arizona": "az",
    "arkansas": "ar",
    "california": "ca",
    "colorado": "co",
    "connecticut": "ct",
    "delaware": "de",
    "florida": "fl",
    "georgia": "ga",
    "hawaii": "hi",
    "idaho": "id",
    "illinois": "il",
    "indiana": "in",
    "iowa": "ia",
    "kansas": "ks",
    "kentucky": "ky",
    "louisiana": "la",
    "maine": "me",
    "maryland": "md",
    "massachusetts": "ma",
    "michigan": "mi",
    "minnesota": "mn",
    "mississippi": "ms",
    "missouri": "mo",
    "montana": "mt",
    "nebraska": "ne",
    "nevada": "nv",
    "new hampshire": "nh",
    "new jersey": "nj",
    "new mexico": "nm",
    "new york": "ny",
    "north carolina": "nc",
    "north dakota": "nd",
    "ohio": "oh",
    "oklahoma": "ok",
    "oregon": "or",
    "pennsylvania": "pa",
    "rhode island": "ri",
    "south carolina": "sc",
    "south dakota": "sd",
    "tennessee": "tn",
    "texas": "tx",
    "utah": "ut",
    "vermont": "vt",
    "virginia": "va",
    "washington": "wa",
    "west virginia": "wv",
    "west va": "wv",
    "wisconsin": "wi",
    "wyoming": "wy",
    "district of columbia": "dc",
    "american samoa": "as",
    "guam": "gu",
    "northern mariana islands": "mp",
    "puerto rico": "pr",
    "united states minor outlying islands": "um",
    "virgin islands": "vi",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))



us_uk_fix = {
    "united states": "us",
    "usa": "us",
    "united kingdom": "england",
}


#canada states

canada_states = {
    'alberta': 'ab',
    'british columbia': 'bc',
    'manitoba': 'mb',
    'new brunswick': 'nb',
    'newfoundland and labrador': 'nl',
    'northwest territories': 'nt',
    'nova scotia': 'ns',
    'nunavut': 'nu',
    'ontario': 'on',
    'prince edward island': 'pe',
    'quebec': 'qc',
    'saskatchewan': 'sk',
    'yukon': 'yt'
}


#DICT of all country names and their continent 
countries_continents = {'africa': ['algeria', 'angola', 'benin', 'botswana', 'british indian ocean territory', 'burkina faso', 'burundi', 'cabo verde', 'cameroon', 'central african republic', 'chad', 'comoros', 'congo', 'côte d’ivoire', 'democratic republic of the congo', 'djibouti', 'egypt', 'equatorial guinea', 'eritrea', 'eswatini', 'ethiopia', 'french southern territories', 'gabon', 'gambia', 'ghana', 'guinea', 'guinea-bissau', 'kenya', 'lesotho', 'liberia', 'libya', 'madagascar', 'malawi', 'mali', 'mauritania', 'mauritius', 'mayotte', 'morocco', 'mozambique', 'namibia', 'niger', 'nigeria', 'réunion', 'rwanda', 'saint helena', 'sao tome and principe', 'senegal', 'seychelles', 'sierra leone', 'somalia', 'south africa', 'south sudan', 'sudan', 'togo', 'tunisia', 'uganda', 'united republic of tanzania', 'western sahara', 'zambia', 'zimbabwe'], 
                        'antarctica': ['antarctica'],
                        'asia': ['afghanistan', 'armenia', 'azerbaijan', 'bahrain', 'bangladesh', 'bhutan', 'brunei darussalam', 'cambodia', 'china', 'hong kong', 'macao', 'cyprus', "democratic people's republic of korea", 'georgia', 'india', 'indonesia', 'iran ', 'iraq', 'israel', 'japan', 'jordan', 'kazakhstan', 'kuwait', 'kyrgyzstan', 'lao ', 'lebanon', 'malaysia', 'maldives', 'mongolia', 'myanmar', 'nepal', 'oman', 'pakistan', 'philippines', 'qatar', 'republic of korea', 'saudi arabia', 'singapore', 'sri lanka', 'state of palestine', 'syrian arab republic', 'tajikistan', 'thailand', 'timor-leste', 'turkey', 'turkmenistan', 'united arab emirates', 'uzbekistan', 'viet nam', 'yemen'],
                        'europe': ['aland islands', 'albania', 'andorra', 'austria', 'belarus', 'belgium', 'bosnia and herzegovina', 'bulgaria', 'croatia', 'czechia', 'denmark', 'estonia', 'faroe islands', 'finland', 'france', 'germany', 'gibraltar', 'greece', 'guernsey', 'holy see', 'hungary', 'iceland', 'ireland', 'isle of man', 'italy', 'jersey', 'latvia', 'liechtenstein', 'lithuania', 'luxembourg', 'malta', 'monaco', 'montenegro', 'netherlands', 'north macedonia', 'norway', 'poland', 'portugal', 'republic of moldova', 'romania', 'russian federation', 'san marino', 'sark', 'serbia', 'slovakia', 'slovenia', 'spain', 'svalbard and jan mayen islands', 'sweden', 'switzerland', 'ukraine', 'united kingdom of great britain and northern ireland'],
                        'north america': ['anguilla', 'antigua and barbuda', 'aruba', 'bahamas', 'barbados', 'belize', 'bermuda', 'bonaire, sint eustatius and saba', 'british virgin islands', 'canada', 'cayman islands', 'costa rica', 'cuba', 'curacao', 'dominica', 'dominican republic', 'el salvador', 'greenland', 'grenada', 'guadeloupe', 'guatemala', 'haiti', 'honduras', 'jamaica', 'martinique', 'mexico', 'montserrat', 'nicaragua', 'panama', 'puerto rico', 'saint barthélemy', 'saint kitts and nevis', 'saint lucia', 'saint martin', 'saint pierre and miquelon', 'saint vincent and the grenadines', 'sint maarten', 'trinidad and tobago', 'turks and caicos islands', 'us', 'united states virgin islands'],
                        'oceania': ['american samoa', 'australia', 'christmas island', 'cocos  islands', 'cook islands', 'fiji', 'french polynesia', 'guam', 'heard island and mcdonald islands', 'kiribati', 'marshall islands', 'micronesia', 'nauru', 'new caledonia', 'new zealand', 'niue', 'norfolk island', 'northern mariana islands', 'palau', 'papua new guinea', 'pitcairn', 'samoa', 'solomon islands', 'tokelau', 'tonga', 'tuvalu', 'united states minor outlying islands', 'vanuatu', 'wallis and futuna islands'],
                        'south america': ['argentina', 'bolivia', 'bouvet island', 'brazil', 'chile', 'colombia', 'ecuador', 'falkland islands', 'french guiana', 'guyana', 'paraguay', 'peru', 'south georgia and the south sandwich islands', 'suriname', 'uruguay', 'venezuela']}

end_time = time.time()

elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.5f} seconds")

Elapsed time: 0.00090 seconds


In [9]:
df = pd.read_csv('hs_uuid_state_country.csv')
origianl_data = pd.read_csv('hs_uuid_state_country.csv')

In [4]:


start_time = time.time()

cols_to_copy = ['uuid', 'State/Region','Country/Region']

# create a new DataFrame with the selected columns
hs_contacts_state_country = df.loc[:, cols_to_copy].copy()


# make lower and clean from punctuations
start_time_lowercase = time.time()
hs_contacts_state_country["State/Region"] = hs_contacts_state_country["State/Region"].astype(str)
hs_contacts_state_country["Country/Region"]= hs_contacts_state_country["Country/Region"].astype(str)


hs_contacts_state_country["State/Region"] = hs_contacts_state_country["State/Region"].str.lower().str.strip()
hs_contacts_state_country["Country/Region"] = hs_contacts_state_country["Country/Region"].str.lower().str.strip()


end_time_lowercase = time.time()
elapsed_time_lowercase = end_time_lowercase - start_time_lowercase
print(f"Elapsed time lowercase: {elapsed_time_lowercase:.5f} seconds")

#letters not in english

start_time_remove_diacritics = time.time()

hs_contacts_state_country["State/Region"] = hs_contacts_state_country["State/Region"].apply(remove_diacritics)
hs_contacts_state_country["Country/Region"] = hs_contacts_state_country["Country/Region"].apply(remove_diacritics)

end_time_remove_diacritics = time.time()
elapsed_time_remove_diacritics = end_time_remove_diacritics - start_time_remove_diacritics
print(f"Elapsed time remove_diacritics: {elapsed_time_remove_diacritics:.5f} seconds")


#change long state name to abv for us and canada
start_time_fix_abv = time.time()

hs_contacts_state_country['State/Region'] = hs_contacts_state_country['State/Region'].replace(us_state_to_abbrev)


hs_contacts_state_country = hs_contacts_state_country.apply(replace_canada_state_name, axis=1)

#clean country - change united states to US, UK/united kingdom to england

hs_contacts_state_country['Country/Region'] = hs_contacts_state_country['Country/Region'].replace(us_uk_fix)

#change all countries to US and Canada if they have a state

hs_contacts_state_country.loc[hs_contacts_state_country['State/Region'].isin(list(us_state_to_abbrev.values())), 'Country/Region'] = 'us'
hs_contacts_state_country.loc[hs_contacts_state_country['State/Region'].isin(list(canada_states.values())), 'Country/Region'] = 'canada'

#if there is a us state in the country change it to us and put the state in the state column

hs_contacts_state_country.loc[hs_contacts_state_country['Country/Region'].isin(list(us_state_to_abbrev.values())), 'Country/Region'] = 'us'


end_time_fix_abv = time.time()
elapsed_time_fix_abv = end_time_fix_abv - start_time_fix_abv
print(f"Elapsed time fix_abv: {elapsed_time_fix_abv:.5f} seconds")

#check if there is a name of a country in the state column
hs_contacts_state_country = hs_contacts_state_country.apply(move_state_name, axis=1)

### create a dict to use for mapping the region - new project




###

end_time = time.time()

#fix nan string in country or state
hs_contacts_state_country['State/Region'] = hs_contacts_state_country['State/Region'].replace('nan', '')
hs_contacts_state_country['Country/Region'] = hs_contacts_state_country['Country/Region'].replace('nan', '')

#Add the continent of each country
hs_contacts_state_country['Continent'] = hs_contacts_state_country['Country/Region'].apply(get_continent)

elapsed_time_total = end_time - start_time
print(f"Elapsed time total: {elapsed_time_total:.5f} seconds")

for column in hs_contacts_state_country.columns:
    num_unique = hs_contacts_state_country[column].nunique()
    print(f"Number of unique values in {column}: {num_unique}")



Elapsed time lowercase: 0.12636 seconds
Elapsed time remove_diacritics: 2.69945 seconds
Elapsed time fix_abv: 9.62404 seconds
Elapsed time total: 17.56184 seconds
Number of unique values in uuid: 99307
Number of unique values in State/Region: 345
Number of unique values in Country/Region: 129
Number of unique values in Continent: 6


In [11]:
for column in df.columns:
    num_unique = df[column].nunique()
    print(f"Number of unique values in {column}: {num_unique}")

Number of unique values in uuid: 99307
Number of unique values in State/Region: 478
Number of unique values in Country/Region: 158


In [10]:
#CHECK THE AFFECTED COLUMNS
df1 = df.copy()

df1["State/Region"] = df1["State/Region"].astype(str)
df1["Country/Region"]= df1["Country/Region"].astype(str)

df1["State/Region"] = df1["State/Region"].str.lower().str.strip()
df1["Country/Region"] = df1["Country/Region"].str.lower().str.strip()

print('When ignoring lowercase, calculating letter changes')
for col in hs_contacts_state_country.columns:
    if col not in df1.columns:
        continue  # skip columns that are not in the old dataframe
    num_changes = (df1[col] != hs_contacts_state_country[col]).sum()
    print(f'Column {col} has {num_changes} cells with changes.')


When ignoring lowercase, calculating letter changes
Column uuid has 0 cells with changes.
Column State/Region has 28310 cells with changes.
Column Country/Region has 70127 cells with changes.


In [12]:
timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
hs_contacts_state_country.to_csv(f'My-Projects/state_country_output/update_state_country_{timestamp}.csv')


OSError: Cannot save file into a non-existent directory: 'My-Projects/state_country_output'

In [ ]:
total_unique_cells = origianl_data["State/Region"].nunique() +  origianl_data["Country/Region"].nunique()

In [133]:
for col in hs_contacts_state_country.columns:
    if col not in origianl_data.columns:
        continue  # skip columns that are not in the old dataframe
    num_changes = (origianl_data[col] != hs_contacts_state_country[col]).sum()
    print(f'Column {col} has {num_changes} cells with changes.')

Column Record ID has 0 cells with changes.
Column State/Region has 98482 cells with changes.
Column Country/Region has 99161 cells with changes.


In [148]:
comparason_df = {
    'dataset': ['original data on hs', 'new output'],
    'unique_record_id': [origianl_data['Record ID'].nunique(), hs_contacts_state_country['Record ID'].nunique()],
    'unique_record_state': [origianl_data["State/Region"].nunique(), hs_contacts_state_country["State/Region"].nunique()],
    'unique_record_country': [origianl_data["Country/Region"].nunique(), hs_contacts_state_country["Country/Region"].nunique()]
}

comparason_df = pd.DataFrame(comparason_df)

In [149]:
comparason_df

,dataset,unique_record_id,unique_record_state,unique_record_country
0,original data on hs,99307,478,158
1,new output,99307,345,129


In [155]:
timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
comparason_df.to_csv(f'comparason_df{timestamp}.csv')

In [101]:
dftest = df

In [106]:
for column in dftest.columns:
    num_unique = dftest[column].nunique()
    print(f"Number of unique values in {column}: {num_unique}")

Number of unique values in Record ID: 99307
Number of unique values in First Name: 14702
Number of unique values in Last Name: 45618
Number of unique values in Email: 95784
Number of unique values in State/Region: 422
Number of unique values in Country/Region: 139


In [135]:
#percent that changed
num_changes_state = (df1["State/Region"] != hs_contacts_state_country["State/Region"]).sum()
num_changes_country = (df1["Country/Region"] != hs_contacts_state_country["Country/Region"]).sum()

In [136]:
num_changes_state

28310

In [137]:
num_changes_country

70127

In [151]:
percente_of_cell_affected_state = (num_changes_state / origianl_data["State/Region"].count())

In [152]:
percente_of_cell_affected_country = (num_changes_country / origianl_data["Country/Region"].count())

In [153]:
affected_cells_df = {
    'type': ['state', 'country'],
    'original_data_row_count': [origianl_data["State/Region"].count(), origianl_data["Country/Region"].count()],
    'new_data_affected_rows': [num_changes_state, num_changes_country],
    'percent_of_affected_rows': [percente_of_cell_affected_state, percente_of_cell_affected_country]
}

affected_cells_df = pd.DataFrame(affected_cells_df)
affected_cells_df

,type,original_data_row_count,new_data_affected_rows,percent_of_affected_rows
0,state,74242,28310,0.381321
1,country,84483,70127,0.830072


In [154]:
affected_cells_df = {
    'type': ['state', 'country'],
    'original_data_row_count': [origianl_data["State/Region"].count(), origianl_data["Country/Region"].count()],
    'new_data_affected_rows': [num_changes_state, num_changes_country],
    'percent_of_affected_rows': [percente_of_cell_affected_state, percente_of_cell_affected_country]
}

affected_cells_df = pd.DataFrame(affected_cells_df)
affected_cells_df

timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
affected_cells_df.to_csv(f'affected_cells_{timestamp}.csv')